# ===============================================================================
# ФИНАЛЬНЫЕ PRODUCTION МОДЕЛИ + SHAP ANALYSIS
# ===============================================================================

**Цель:** Воспроизводимый notebook с лучшими моделями для каждого сегмента

**Содержание:**
1. Загрузка подготовленных данных (parquet из output/)
2. **Статистика ABT (Section 3.3)** - для документации
3. **PSI Index (Section 3.5.3)** - Population Stability Index
4. **Корреляционный анализ (Section 3.5.4)** - Multicollinearity
5. Конфигурация лучших моделей (HARDCODED из экспериментов 02-03)
6. **Сравнение моделей (Section 4.2)** - ROC-AUC/Gini visualization
7. Обучение финальных моделей
8. **SHAP Analysis (Section 5.3)** - Feature Importance + Explainability
9. Полные метрики и визуализации
10. Сохранение финальных моделей

**Reproducibility:** Random seed = 42, Run All должен давать те же результаты

**Standalone:** Требуются только parquet файлы из output/, все модели прописаны в коде

**Дата:** 2025-01-13

# ===============================================================================

---
# 1. ИМПОРТ БИБЛИОТЕК И КОНФИГУРАЦИЯ

In [ ]:
# ====================================================================================
# ИМПОРТ БИБЛИОТЕК
# ====================================================================================

import os
import warnings
from datetime import datetime
from pathlib import Path
import pickle
import time

# Данные
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# ML Models
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# Balancing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight

# Metrics
from sklearn.metrics import (
    roc_auc_score, roc_curve, average_precision_score,
    precision_score, recall_score, f1_score, confusion_matrix,
    classification_report
)

# SHAP for explainability
import shap

# Настройки
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("="*80)
print("ФИНАЛЬНЫЕ PRODUCTION МОДЕЛИ + SHAP ANALYSIS")
print("="*80)
print(f"✓ Библиотеки импортированы")
print(f"  Дата запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [ ]:
# ====================================================================================
# КОНФИГУРАЦИЯ
# ====================================================================================

class Config:
    """Централизованная конфигурация"""
    
    # ВОСПРОИЗВОДИМОСТЬ
    RANDOM_SEED = 42
    
    # ПУТИ
    OUTPUT_DIR = Path("output")
    MODELS_DIR = Path("models")
    FIGURES_DIR = Path("figures")
    
    # КОЛОНКИ
    TARGET_COLUMN = 'target_churn_3m'
    
    # СЕГМЕНТЫ
    SEGMENTS = {
        'Segment 1': {
            'name': 'Small Business',
            'train': 'seg1_train.parquet',
            'val': 'seg1_val.parquet',
            'test': 'seg1_test.parquet'
        },
        'Segment 2': {
            'name': 'Middle + Large Business',
            'train': 'seg2_train.parquet',
            'val': 'seg2_val.parquet',
            'test': 'seg2_test.parquet'
        }
    }
    
    # SHAP PARAMETERS
    SHAP_SAMPLE_SIZE = 100  # Для Tree SHAP используем все данные или sample
    SHAP_TOP_FEATURES = 20  # Топ признаков для визуализации
    
    # ВИЗУАЛИЗАЦИЯ
    FIGURE_SIZE = (12, 8)
    FIGURE_DPI = 100
    
    @classmethod
    def create_directories(cls):
        for dir_path in [cls.OUTPUT_DIR, cls.MODELS_DIR, cls.FIGURES_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)

config = Config()
config.create_directories()
np.random.seed(config.RANDOM_SEED)

print("\n✓ Конфигурация инициализирована")
print(f"  Random seed: {config.RANDOM_SEED}")
print(f"  Сегментов: {len(config.SEGMENTS)}")

---
# 2. ЗАГРУЗКА ДАННЫХ

In [ ]:
# ====================================================================================
# ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ
# ====================================================================================

print("\n" + "="*80)
print("ЗАГРУЗКА ДАННЫХ")
print("="*80)

data = {}

for seg_id, seg_info in config.SEGMENTS.items():
    print(f"\n{seg_id}: {seg_info['name']}")
    print("-" * 80)
    
    data[seg_id] = {}
    
    for split in ['train', 'val', 'test']:
        file_path = config.OUTPUT_DIR / seg_info[split]
        
        if not file_path.exists():
            raise FileNotFoundError(
                f"Файл не найден: {file_path}\n"
                f"Сначала запустите notebook 01_data_preparation_eda.ipynb"
            )
        
        df = pd.read_parquet(file_path)
        data[seg_id][split] = df
        
        churn_rate = df[config.TARGET_COLUMN].mean()
        print(f"  {split.upper():5s}: {df.shape} | Churn: {churn_rate*100:.2f}%")

print("\n" + "="*80)
print("✓ Все данные загружены успешно")
print("="*80)

---
# 2.1. СТАТИСТИКА ABT (Section 3.3 - Для документации)

In [ ]:
# ====================================================================================
# СТАТИСТИКА ABT (ANALYTICAL BASE TABLE)
# ====================================================================================
# Раздел 3.3 документации: Результаты сбора ABT

print("\n" + "="*80)
print("СТАТИСТИКА ИТОГОВОЙ ВИТРИНЫ ABT")
print("="*80)

abt_statistics = {}

for seg_id, seg_info in config.SEGMENTS.items():
    print(f"\n{seg_id}: {seg_info['name']}")
    print("="*80)
    
    # Объединяем все splits для полной статистики
    train_df = data[seg_id]['train']
    val_df = data[seg_id]['val']
    test_df = data[seg_id]['test']
    
    full_df = pd.concat([train_df, val_df, test_df], axis=0)
    
    # Разделяем на числовые и не числовые
    numeric_cols = full_df.select_dtypes(include=[np.number]).columns.tolist()
    non_numeric_cols = full_df.select_dtypes(exclude=[np.number]).columns.tolist()
    
    # Убираем target из предикторов
    if config.TARGET_COLUMN in numeric_cols:
        numeric_cols.remove(config.TARGET_COLUMN)
    
    stats = {
        'Количество наблюдений': len(full_df),
        'Количество событий (churn=1)': full_df[config.TARGET_COLUMN].sum(),
        'Уровень целевой переменной (%)': f"{full_df[config.TARGET_COLUMN].mean()*100:.2f}%",
        'Количество числовых предикторов': len(numeric_cols),
        'Количество не числовых предикторов': len(non_numeric_cols),
        'Всего признаков': len(numeric_cols) + len(non_numeric_cols),
        'Train размер': len(train_df),
        'Val размер': len(val_df),
        'Test размер': len(test_df)
    }
    
    abt_statistics[seg_id] = stats
    
    # Вывод таблицы
    print("\nСтатистика итоговой витрины ABT:")
    print("-" * 80)
    for key, value in stats.items():
        print(f"  {key:45s}: {value}")
    
    print("\n" + "-" * 80)

print("\n" + "="*80)
print("✓ Статистика ABT рассчитана для всех сегментов")
print("="*80)

# Сводная таблица для документации
print("\n" + "="*80)
print("СВОДНАЯ ТАБЛИЦА ABT (для Section 3.3)")
print("="*80)

abt_summary = pd.DataFrame(abt_statistics).T
abt_summary.index.name = 'Segment'
print("\n" + abt_summary.to_string())
print("\n" + "="*80)

---
# 2.2. PSI INDEX (Section 3.5.3 - Population Stability Index)

In [ ]:
# ====================================================================================
# PSI (POPULATION STABILITY INDEX)
# ====================================================================================
# Раздел 3.5.3 документации: Индекс PSI

def calculate_psi(expected, actual, bins=10):
    """
    Рассчитать PSI (Population Stability Index) между двумя распределениями.
    
    PSI < 0.1  - Отличная стабильность
    PSI < 0.2  - Приемлемая стабильность  
    PSI >= 0.2 - Значительное изменение, требует внимания
    """
    # Объединяем для определения границ бинов
    combined = np.concatenate([expected, actual])
    min_val = combined.min()
    max_val = combined.max()
    
    # Создаем бины
    breakpoints = np.linspace(min_val, max_val, bins + 1)
    breakpoints[0] = -np.inf
    breakpoints[-1] = np.inf
    
    # Распределения по бинам
    expected_counts = np.histogram(expected, bins=breakpoints)[0]
    actual_counts = np.histogram(actual, bins=breakpoints)[0]
    
    # Пропорции (с защитой от деления на 0)
    expected_percents = expected_counts / len(expected)
    actual_percents = actual_counts / len(actual)
    
    # Защита от нулевых значений (добавляем малое число)
    expected_percents = np.where(expected_percents == 0, 0.0001, expected_percents)
    actual_percents = np.where(actual_percents == 0, 0.0001, actual_percents)
    
    # PSI формула
    psi_values = (actual_percents - expected_percents) * np.log(actual_percents / expected_percents)
    psi = np.sum(psi_values)
    
    return psi

def interpret_psi(psi_value):
    """Интерпретация значения PSI"""
    if psi_value < 0.1:
        return "✅ Отлично - модель стабильна"
    elif psi_value < 0.2:
        return "⚠️  Приемлемо - небольшие изменения"
    else:
        return "❌ Требует внимания - значительный drift"

print("\n" + "="*80)
print("PSI (POPULATION STABILITY INDEX) - TRAIN vs TEST")
print("="*80)

psi_results = {}

for seg_id in config.SEGMENTS.keys():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("="*80)
    
    # Получаем данные без использования prepare_data
    train_df = data[seg_id]['train']
    test_df = data[seg_id]['test']
    
    X_train = train_df.drop(columns=[config.TARGET_COLUMN])
    X_test = test_df.drop(columns=[config.TARGET_COLUMN])
    
    feature_psi = {}
    
    # Рассчитываем PSI для каждого признака
    for col in X_train.columns:
        try:
            psi_val = calculate_psi(X_train[col].values, X_test[col].values)
            feature_psi[col] = psi_val
        except Exception as e:
            feature_psi[col] = np.nan
    
    # Создаем DataFrame
    psi_df = pd.DataFrame({
        'Feature': list(feature_psi.keys()),
        'PSI': list(feature_psi.values())
    }).sort_values('PSI', ascending=False).reset_index(drop=True)
    
    psi_df['Interpretation'] = psi_df['PSI'].apply(interpret_psi)
    
    # Overall PSI (среднее по всем признакам)
    overall_psi = psi_df['PSI'].mean()
    
    print(f"\nОбщий PSI (среднее по всем признакам): {overall_psi:.6f}")
    print(f"Интерпретация: {interpret_psi(overall_psi)}")
    
    print("\n" + "-" * 80)
    print("ТОП-15 признаков с наибольшим PSI:")
    print("-" * 80)
    print(psi_df.head(15).to_string(index=False))
    
    # Статистика по категориям PSI
    excellent = (psi_df['PSI'] < 0.1).sum()
    acceptable = ((psi_df['PSI'] >= 0.1) & (psi_df['PSI'] < 0.2)).sum()
    concerning = (psi_df['PSI'] >= 0.2).sum()
    
    print("\n" + "-" * 80)
    print("Распределение признаков по PSI:")
    print(f"  ✅ Отличная стабильность (PSI < 0.1):     {excellent} признаков ({excellent/len(psi_df)*100:.1f}%)")
    print(f"  ⚠️  Приемлемая стабильность (0.1-0.2):    {acceptable} признаков ({acceptable/len(psi_df)*100:.1f}%)")
    print(f"  ❌ Требует внимания (PSI >= 0.2):         {concerning} признаков ({concerning/len(psi_df)*100:.1f}%)")
    
    psi_results[seg_id] = {
        'overall_psi': overall_psi,
        'psi_df': psi_df,
        'excellent': excellent,
        'acceptable': acceptable,
        'concerning': concerning
    }
    
    # Сохраняем
    seg_num = seg_id.split()[1]
    psi_file = config.OUTPUT_DIR / f'psi_analysis_seg{seg_num}.csv'
    psi_df.to_csv(psi_file, index=False)
    print(f"\n✓ Сохранено: {psi_file}")

print("\n" + "="*80)
print("✓ PSI анализ завершен для всех сегментов")
print("="*80)

---
# 2.3. КОРРЕЛЯЦИОННЫЙ АНАЛИЗ И МУЛЬТИКОЛЛИНЕАРНОСТЬ (Section 3.5.4)

In [ ]:
# ====================================================================================
# КОРРЕЛЯЦИОННЫЙ АНАЛИЗ И МУЛЬТИКОЛЛИНЕАРНОСТЬ
# ====================================================================================
# Раздел 3.5.4 документации: Корреляционный анализ и мультиколлинеарность

print("\n" + "="*80)
print("КОРРЕЛЯЦИОННЫЙ АНАЛИЗ И МУЛЬТИКОЛЛИНЕАРНОСТЬ")
print("="*80)

correlation_results = {}

for seg_id in config.SEGMENTS.keys():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("="*80)
    
    # Получаем данные без использования prepare_data
    train_df = data[seg_id]['train']
    X_train = train_df.drop(columns=[config.TARGET_COLUMN])
    y_train = train_df[config.TARGET_COLUMN]
    
    # Корреляционная матрица
    corr_matrix = X_train.corr()
    
    # 1. Корреляция с целевой переменной
    print("\n1. КОРРЕЛЯЦИЯ ПРИЗНАКОВ С ЦЕЛЕВОЙ ПЕРЕМЕННОЙ")
    print("-" * 80)
    
    # Добавляем target для расчета корреляции
    temp_df = X_train.copy()
    temp_df[config.TARGET_COLUMN] = y_train
    
    target_corr = temp_df.corr()[config.TARGET_COLUMN].drop(config.TARGET_COLUMN).abs().sort_values(ascending=False)
    
    print("\nТОП-20 признаков с наибольшей корреляцией с target:")
    print(target_corr.head(20).to_string())
    
    # 2. Мультиколлинеарность (высокая корреляция между признаками)
    print("\n\n2. МУЛЬТИКОЛЛИНЕАРНОСТЬ (высокая корреляция между признаками)")
    print("-" * 80)
    
    # Находим пары признаков с высокой корреляцией (> 0.8)
    high_corr_threshold = 0.8
    high_corr_pairs = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > high_corr_threshold:
                high_corr_pairs.append({
                    'Feature_1': corr_matrix.columns[i],
                    'Feature_2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    
    if high_corr_pairs:
        high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False, key=abs)
        print(f"\nНайдено {len(high_corr_df)} пар признаков с |correlation| > {high_corr_threshold}:")
        print("\nТОП-20 пар с наибольшей корреляцией:")
        print(high_corr_df.head(20).to_string(index=False))
    else:
        print(f"\n✅ Пар признаков с |correlation| > {high_corr_threshold} не найдено")
        print("   Мультиколлинеарность не является проблемой")
    
    # 3. Визуализация корреляционной матрицы (топ-30 признаков по корреляции с target)
    print("\n\n3. ВИЗУАЛИЗАЦИЯ КОРРЕЛЯЦИОННОЙ МАТРИЦЫ")
    print("-" * 80)
    
    # Берем топ-30 признаков
    top_features = target_corr.head(30).index.tolist()
    
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # Матрица корреляции для топ признаков
    top_corr_matrix = X_train[top_features].corr()
    
    sns.heatmap(
        top_corr_matrix,
        cmap='coolwarm',
        center=0,
        annot=False,
        fmt='.2f',
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.8},
        ax=ax
    )
    
    ax.set_title(f'Корреляционная матрица (ТОП-30 признаков) - {seg_id}', 
                 fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    
    seg_num = seg_id.split()[1]
    corr_plot_path = config.FIGURES_DIR / f'correlation_matrix_seg{seg_num}.png'
    plt.savefig(corr_plot_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Сохранено: {corr_plot_path}")
    
    # 4. Сохранение результатов
    correlation_results[seg_id] = {
        'target_correlation': target_corr,
        'high_corr_pairs': high_corr_df if high_corr_pairs else None,
        'corr_matrix': corr_matrix
    }
    
    # Сохраняем CSV с корреляциями
    target_corr_file = config.OUTPUT_DIR / f'target_correlation_seg{seg_num}.csv'
    target_corr.to_csv(target_corr_file, header=['Correlation_with_Target'])
    print(f"✓ Сохранено: {target_corr_file}")
    
    if high_corr_pairs:
        high_corr_file = config.OUTPUT_DIR / f'high_correlation_pairs_seg{seg_num}.csv'
        high_corr_df.to_csv(high_corr_file, index=False)
        print(f"✓ Сохранено: {high_corr_file}")

print("\n" + "="*80)
print("✓ Корреляционный анализ завершен для всех сегментов")
print("="*80)

---
# 3. КОНФИГУРАЦИЯ ЛУЧШИХ МОДЕЛЕЙ

**Модели выбраны по результатам экспериментов (notebooks 02-03):**
- **Segment 1 (Small Business):** XGBoost + No balancing → ROC-AUC 0.8958
- **Segment 2 (Middle + Large Business):** CatBoost + No balancing → ROC-AUC 0.8768

In [ ]:
# ====================================================================================
# КОНФИГУРАЦИЯ ЛУЧШИХ МОДЕЛЕЙ (HARDCODED)
# ====================================================================================

print("\n" + "="*80)
print("КОНФИГУРАЦИЯ ЛУЧШИХ МОДЕЛЕЙ")
print("="*80)

# ============================================================================
# ЛУЧШИЕ МОДЕЛИ ПО РЕЗУЛЬТАТАМ ЭКСПЕРИМЕНТОВ
# ============================================================================
# Segment 1 (Small Business):     XGBoost + No balancing = ROC-AUC 0.8958
# Segment 2 (Middle + Large):      CatBoost + No balancing = ROC-AUC 0.8768
# ============================================================================

best_models_config = {
    'Segment 1': {
        'algorithm': 'XGBoost',
        'balancing_method': 'No balancing',
        'threshold': 0.12,
        'expected_roc_auc': 0.8958,  # Ожидаемый результат из экспериментов
        'description': 'Small Business - XGBoost показал лучшую discriminative power'
    },
    'Segment 2': {
        'algorithm': 'CatBoost',
        'balancing_method': 'No balancing',
        'threshold': 0.10,
        'expected_roc_auc': 0.8768,  # Ожидаемый результат из экспериментов
        'description': 'Middle + Large Business - CatBoost обеспечил высокую стабильность'
    }
}

print("\n✓ Конфигурация лучших моделей (из экспериментов 02-03):")
print("="*80)

for seg_id, model_config in best_models_config.items():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    print(f"  Алгоритм:       {model_config['algorithm']}")
    print(f"  Балансировка:   {model_config['balancing_method']}")
    print(f"  Threshold:      {model_config['threshold']:.2f}")
    print(f"  Ожидаемый AUC:  {model_config['expected_roc_auc']:.4f}")
    print(f"  Описание:       {model_config['description']}")

print("\n" + "="*80)
print("✓ Модели готовы к обучению")
print("="*80)

---
# 4. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ

In [ ]:
# ====================================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ====================================================================================

def prepare_data(df, target_col):
    """Разделение на X и y"""
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return X, y


def apply_balancing(X_train, y_train, method, random_seed=42):
    """
    Применить метод балансировки.
    """
    if method == 'No balancing':
        return X_train.copy(), y_train.copy(), None
    
    elif method == 'Class weights':
        class_weights = compute_class_weight(
            'balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        sample_weights = np.array([class_weights[int(y)] for y in y_train])
        return X_train.copy(), y_train.copy(), sample_weights
    
    elif method == 'SMOTE':
        smote = SMOTE(random_state=random_seed, k_neighbors=5)
        X_res, y_res = smote.fit_resample(X_train, y_train)
        return X_res, y_res, None
    
    elif method == 'Random Undersampling':
        rus = RandomUnderSampler(random_state=random_seed)
        X_res, y_res = rus.fit_resample(X_train, y_train)
        return X_res, y_res, None
    
    elif method == 'SMOTE + Undersampling':
        smote = SMOTE(random_state=random_seed, k_neighbors=5)
        X_smote, y_smote = smote.fit_resample(X_train, y_train)
        rus = RandomUnderSampler(random_state=random_seed)
        X_res, y_res = rus.fit_resample(X_smote, y_smote)
        return X_res, y_res, None
    
    else:
        raise ValueError(f"Unknown balancing method: {method}")


def train_model(algorithm, X_train, y_train, X_val, y_val, sample_weights=None, random_seed=42):
    """
    Обучить модель с оптимальными параметрами.
    """
    if algorithm == 'CatBoost':
        model = CatBoostClassifier(
            iterations=300,
            depth=6,
            learning_rate=0.05,
            loss_function='Logloss',
            eval_metric='AUC',
            early_stopping_rounds=50,
            use_best_model=True,
            random_seed=random_seed,
            task_type='CPU',
            verbose=False,
            allow_writing_files=False
        )
        from catboost import Pool
        train_pool = Pool(X_train, y_train, weight=sample_weights)
        val_pool = Pool(X_val, y_val)
        model.fit(train_pool, eval_set=val_pool)
        
    elif algorithm == 'LightGBM':
        model = LGBMClassifier(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            objective='binary',
            metric='auc',
            random_state=random_seed,
            verbose=-1,
            n_jobs=-1
        )
        model.fit(
            X_train, y_train,
            sample_weight=sample_weights,
            eval_set=[(X_val, y_val)],
            callbacks=[]
        )
        
    elif algorithm == 'XGBoost':
        model = XGBClassifier(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            objective='binary:logistic',
            eval_metric='auc',
            early_stopping_rounds=50,
            random_state=random_seed,
            n_jobs=-1,
            verbosity=0
        )
        model.fit(
            X_train, y_train,
            sample_weight=sample_weights,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        
    elif algorithm == 'RandomForest':
        model = RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            min_samples_split=100,
            random_state=random_seed,
            n_jobs=-1,
            verbose=0
        )
        model.fit(X_train, y_train, sample_weight=sample_weights)
    
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")
    
    return model


def calculate_metrics(y_true, y_pred_proba, threshold):
    """
    Рассчитать все метрики.
    """
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    metrics = {
        'threshold': threshold,
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'pr_auc': average_precision_score(y_true, y_pred_proba),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
    }
    
    metrics['gini'] = 2 * metrics['roc_auc'] - 1
    
    cm = confusion_matrix(y_true, y_pred)
    metrics['tn'] = cm[0, 0]
    metrics['fp'] = cm[0, 1]
    metrics['fn'] = cm[1, 0]
    metrics['tp'] = cm[1, 1]
    
    return metrics


print("✓ Вспомогательные функции определены")

---
# 4.2. СРАВНЕНИЕ МОДЕЛЕЙ (Section 4.2 - Выбор лучшей модели)

**Цель:** Показать обоснование выбора лучших моделей для каждого сегмента

Проведем быстрое сравнение разных алгоритмов на базовых настройках для визуализации.

In [ ]:
# ====================================================================================
# СРАВНЕНИЕ АЛГОРИТМОВ - БЫСТРАЯ ОЦЕНКА
# ====================================================================================
# Раздел 4.2 документации: Выбор лучшей модели

print("\n" + "="*80)
print("СРАВНЕНИЕ АЛГОРИТМОВ (для обоснования выбора)")
print("="*80)
print("Проводим быстрое обучение всех алгоритмов с базовыми параметрами...")
print("="*80)

algorithms_to_compare = ['CatBoost', 'LightGBM', 'XGBoost', 'RandomForest']
comparison_results = {}

for seg_id in config.SEGMENTS.keys():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    
    # Подготовка данных
    X_train, y_train = prepare_data(data[seg_id]['train'], config.TARGET_COLUMN)
    X_val, y_val = prepare_data(data[seg_id]['val'], config.TARGET_COLUMN)
    X_test, y_test = prepare_data(data[seg_id]['test'], config.TARGET_COLUMN)
    
    seg_results = []
    
    for algo in algorithms_to_compare:
        print(f"  {algo}...", end=' ')
        
        try:
            # Обучаем с базовыми параметрами (без балансировки)
            model = train_model(
                algo,
                X_train, y_train,
                X_val, y_val,
                sample_weights=None,
                random_seed=config.RANDOM_SEED
            )
            
            # Предсказания на test
            y_test_proba = model.predict_proba(X_test)[:, 1]
            
            # Метрики
            roc_auc = roc_auc_score(y_test, y_test_proba)
            gini = 2 * roc_auc - 1
            pr_auc = average_precision_score(y_test, y_test_proba)
            
            seg_results.append({
                'Algorithm': algo,
                'ROC_AUC': roc_auc,
                'Gini': gini,
                'PR_AUC': pr_auc
            })
            
            print(f"ROC-AUC: {roc_auc:.4f}, Gini: {gini:.4f}")
            
        except Exception as e:
            print(f"ERROR: {str(e)[:50]}")
            seg_results.append({
                'Algorithm': algo,
                'ROC_AUC': 0,
                'Gini': 0,
                'PR_AUC': 0
            })
    
    comparison_results[seg_id] = pd.DataFrame(seg_results).sort_values('ROC_AUC', ascending=False)
    
    print("\n" + "-" * 80)
    print("Результаты сравнения:")
    print(comparison_results[seg_id].to_string(index=False))

print("\n" + "="*80)
print("✓ Сравнение алгоритмов завершено")
print("="*80)

In [ ]:
# ====================================================================================
# ВИЗУАЛИЗАЦИЯ СРАВНЕНИЯ МОДЕЛЕЙ
# ====================================================================================

print("\n" + "="*80)
print("ВИЗУАЛИЗАЦИЯ: СРАВНЕНИЕ ROC-AUC И GINI ПО АЛГОРИТМАМ")
print("="*80)

# Создаем сравнительные графики
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('СРАВНЕНИЕ АЛГОРИТМОВ ПО СЕГМЕНТАМ\n(Обоснование выбора лучших моделей)', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, (seg_id, results_df) in enumerate(comparison_results.items()):
    # ROC-AUC график
    ax_roc = axes[idx, 0]
    bars_roc = ax_roc.barh(results_df['Algorithm'], results_df['ROC_AUC'], 
                           color=['#2E86AB' if algo == best_models_config[seg_id]['algorithm'] 
                                  else '#A23B72' for algo in results_df['Algorithm']])
    
    ax_roc.set_xlabel('ROC-AUC', fontsize=11, fontweight='bold')
    ax_roc.set_title(f'{seg_id}: ROC-AUC Comparison', fontsize=12, fontweight='bold')
    ax_roc.set_xlim([0, 1])
    ax_roc.grid(axis='x', alpha=0.3)
    
    # Добавляем значения на бары
    for bar in bars_roc:
        width = bar.get_width()
        ax_roc.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                   f'{width:.4f}', ha='left', va='center', fontsize=10, fontweight='bold')
    
    # Отмечаем лучшую модель
    best_algo = best_models_config[seg_id]['algorithm']
    ax_roc.text(0.02, 0.98, f'✓ ВЫБРАНО: {best_algo}', 
               transform=ax_roc.transAxes, fontsize=10, fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
               verticalalignment='top')
    
    # Gini график
    ax_gini = axes[idx, 1]
    bars_gini = ax_gini.barh(results_df['Algorithm'], results_df['Gini'],
                            color=['#2E86AB' if algo == best_models_config[seg_id]['algorithm'] 
                                   else '#A23B72' for algo in results_df['Algorithm']])
    
    ax_gini.set_xlabel('Gini Coefficient', fontsize=11, fontweight='bold')
    ax_gini.set_title(f'{seg_id}: Gini Comparison', fontsize=12, fontweight='bold')
    ax_gini.set_xlim([0, 1])
    ax_gini.grid(axis='x', alpha=0.3)
    
    # Добавляем значения на бары
    for bar in bars_gini:
        width = bar.get_width()
        ax_gini.text(width + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{width:.4f}', ha='left', va='center', fontsize=10, fontweight='bold')
    
    # Отмечаем лучшую модель
    ax_gini.text(0.02, 0.98, f'✓ ВЫБРАНО: {best_algo}',
                transform=ax_gini.transAxes, fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7),
                verticalalignment='top')

plt.tight_layout()

comparison_plot_path = config.FIGURES_DIR / 'model_comparison_roc_gini.png'
plt.savefig(comparison_plot_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
plt.show()

print(f"\n✓ Сохранено: {comparison_plot_path}")
print("\n" + "="*80)
print("ВЫВОДЫ:")
print("="*80)
for seg_id in comparison_results.keys():
    best_algo = best_models_config[seg_id]['algorithm']
    best_roc = comparison_results[seg_id][comparison_results[seg_id]['Algorithm'] == best_algo]['ROC_AUC'].values[0]
    print(f"\n{seg_id}:")
    print(f"  ✓ Выбрана модель: {best_algo}")
    print(f"  ✓ ROC-AUC: {best_roc:.4f}")
    print(f"  ✓ Обоснование: Показала наилучшую discriminative power в экспериментах")
print("\n" + "="*80)

---
# 5. ОБУЧЕНИЕ ЛУЧШИХ МОДЕЛЕЙ

In [ ]:
# ====================================================================================
# ОБУЧЕНИЕ ЛУЧШИХ МОДЕЛЕЙ ДЛЯ КАЖДОГО СЕГМЕНТА
# ====================================================================================

print("\n" + "="*80)
print("ОБУЧЕНИЕ ЛУЧШИХ МОДЕЛЕЙ")
print("="*80)

final_models = {}
final_results = {}

for seg_id, model_config in best_models_config.items():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    print(f"  Алгоритм: {model_config['algorithm']}")
    print(f"  Балансировка: {model_config['balancing_method']}")
    
    # Подготовка данных
    X_train, y_train = prepare_data(data[seg_id]['train'], config.TARGET_COLUMN)
    X_val, y_val = prepare_data(data[seg_id]['val'], config.TARGET_COLUMN)
    X_test, y_test = prepare_data(data[seg_id]['test'], config.TARGET_COLUMN)
    
    # Применение балансировки
    X_train_balanced, y_train_balanced, sample_weights = apply_balancing(
        X_train, y_train, 
        model_config['balancing_method'],
        config.RANDOM_SEED
    )
    
    print(f"\n  Размер после балансировки: {X_train_balanced.shape}")
    print(f"  Churn rate: {y_train_balanced.mean()*100:.2f}%")
    
    # Обучение модели
    print(f"\n  Обучение модели...")
    start_time = time.time()
    
    model = train_model(
        model_config['algorithm'],
        X_train_balanced, y_train_balanced,
        X_val, y_val,
        sample_weights,
        config.RANDOM_SEED
    )
    
    train_time = time.time() - start_time
    print(f"  ✓ Обучение завершено за {train_time:.2f} сек")
    
    # Предсказания
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Метрики
    threshold = model_config.get('threshold', 0.5)
    metrics = calculate_metrics(y_test, y_test_proba, threshold)
    
    print(f"\n  МЕТРИКИ НА TEST:")
    print(f"    ROC-AUC:   {metrics['roc_auc']:.4f}")
    print(f"    Gini:      {metrics['gini']:.4f}")
    print(f"    F1-Score:  {metrics['f1']:.4f}")
    print(f"    Precision: {metrics['precision']:.4f}")
    print(f"    Recall:    {metrics['recall']:.4f}")
    print(f"    Threshold: {threshold:.4f}")
    
    # Сохранение
    final_models[seg_id] = {
        'model': model,
        'X_train': X_train,  # Для SHAP
        'X_test': X_test,
        'y_test': y_test,
        'y_test_proba': y_test_proba,
        'feature_names': X_train.columns.tolist()
    }
    
    final_results[seg_id] = {
        'algorithm': model_config['algorithm'],
        'balancing_method': model_config['balancing_method'],
        'train_time': train_time,
        **metrics
    }

print("\n" + "="*80)
print("✓ Все модели обучены")
print("="*80)

---
# 6. SHAP ANALYSIS - EXPLAINABILITY (Section 5.3)

In [ ]:
# ====================================================================================
# SHAP ANALYSIS ДЛЯ ЛУЧШИХ МОДЕЛЕЙ
# ====================================================================================

print("\n" + "="*80)
print("SHAP ANALYSIS - EXPLAINABILITY")
print("="*80)

shap_values_storage = {}

for seg_id, model_data in final_models.items():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    
    model = model_data['model']
    X_test = model_data['X_test']
    
    # Используем sample для SHAP если данных много
    if len(X_test) > config.SHAP_SAMPLE_SIZE:
        print(f"  Используем sample {config.SHAP_SAMPLE_SIZE} из {len(X_test)} для SHAP")
        X_shap = X_test.sample(n=config.SHAP_SAMPLE_SIZE, random_state=config.RANDOM_SEED)
    else:
        X_shap = X_test
    
    print(f"  Расчет SHAP values...")
    start_time = time.time()
    
    try:
        # Tree SHAP для tree-based моделей
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_shap)
        
        # Для бинарной классификации берем SHAP values для класса 1
        if isinstance(shap_values, list):
            shap_values = shap_values[1]
        
        elapsed = time.time() - start_time
        print(f"  ✓ SHAP values рассчитаны за {elapsed:.2f} сек")
        
        shap_values_storage[seg_id] = {
            'explainer': explainer,
            'shap_values': shap_values,
            'X_shap': X_shap
        }
        
    except Exception as e:
        print(f"  ❌ Ошибка при расчете SHAP: {str(e)}")
        shap_values_storage[seg_id] = None

print("\n" + "="*80)
print("✓ SHAP analysis завершен")
print("="*80)

In [ ]:
# ====================================================================================
# SHAP SUMMARY PLOTS
# ====================================================================================

print("\n" + "="*80)
print("SHAP SUMMARY PLOTS")
print("="*80)

for seg_id, shap_data in shap_values_storage.items():
    if shap_data is None:
        continue
    
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    
    shap_values = shap_data['shap_values']
    X_shap = shap_data['X_shap']
    
    # 1. Summary Plot (Bar) - Feature Importance
    fig, ax = plt.subplots(figsize=(12, 8))
    shap.summary_plot(
        shap_values, 
        X_shap,
        plot_type="bar",
        max_display=config.SHAP_TOP_FEATURES,
        show=False
    )
    plt.title(f'SHAP Feature Importance: {seg_id}', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    
    seg_num = seg_id.split()[1]
    bar_path = config.FIGURES_DIR / f'shap_importance_seg{seg_num}.png'
    plt.savefig(bar_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
    plt.show()
    print(f"  ✓ Сохранено: {bar_path}")
    
    # 2. Summary Plot (Beeswarm) - Feature Impact
    fig, ax = plt.subplots(figsize=(12, 10))
    shap.summary_plot(
        shap_values,
        X_shap,
        max_display=config.SHAP_TOP_FEATURES,
        show=False
    )
    plt.title(f'SHAP Feature Impact: {seg_id}', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    
    beeswarm_path = config.FIGURES_DIR / f'shap_beeswarm_seg{seg_num}.png'
    plt.savefig(beeswarm_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
    plt.show()
    print(f"  ✓ Сохранено: {beeswarm_path}")

print("\n" + "="*80)

In [ ]:
# ====================================================================================
# SHAP FEATURE IMPORTANCE TABLE
# ====================================================================================

print("\n" + "="*80)
print("SHAP FEATURE IMPORTANCE (ТОП-20)")
print("="*80)

for seg_id, shap_data in shap_values_storage.items():
    if shap_data is None:
        continue
    
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    
    shap_values = shap_data['shap_values']
    X_shap = shap_data['X_shap']
    
    # Рассчитываем mean absolute SHAP values
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    
    # Создаем DataFrame
    importance_df = pd.DataFrame({
        'Feature': X_shap.columns,
        'SHAP_Importance': mean_abs_shap
    }).sort_values('SHAP_Importance', ascending=False).reset_index(drop=True)
    
    # Топ-20
    top20 = importance_df.head(20)
    print("\nТОП-20 признаков по SHAP importance:")
    print(top20.to_string(index=False))
    
    # Сохраняем
    seg_num = seg_id.split()[1]
    importance_file = config.OUTPUT_DIR / f'shap_importance_seg{seg_num}.csv'
    importance_df.to_csv(importance_file, index=False)
    print(f"\n✓ Сохранено: {importance_file}")

print("\n" + "="*80)

---
# 7. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ

In [ ]:
# ====================================================================================
# ROC CURVES
# ====================================================================================

print("\n" + "="*80)
print("ROC CURVES")
print("="*80)

fig, ax = plt.subplots(figsize=(10, 8))

colors = ['#2E86AB', '#A23B72']

for idx, (seg_id, model_data) in enumerate(final_models.items()):
    y_test = model_data['y_test']
    y_test_proba = model_data['y_test_proba']
    
    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    roc_auc = final_results[seg_id]['roc_auc']
    algorithm = final_results[seg_id]['algorithm']
    
    label = f"{seg_id} | {algorithm} (AUC = {roc_auc:.4f})"
    ax.plot(fpr, tpr, color=colors[idx], lw=2, label=label)

# Diagonal
ax.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random (AUC = 0.5000)')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
ax.set_title('ROC CURVES - ФИНАЛЬНЫЕ МОДЕЛИ', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
roc_path = config.FIGURES_DIR / 'final_roc_curves.png'
plt.savefig(roc_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
plt.show()

print(f"\n✓ Сохранено: {roc_path}")
print("="*80)

In [ ]:
# ====================================================================================
# CONFUSION MATRICES
# ====================================================================================

print("\n" + "="*80)
print("CONFUSION MATRICES")
print("="*80)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, (seg_id, result) in enumerate(final_results.items()):
    cm = np.array([[result['tn'], result['fp']], 
                   [result['fn'], result['tp']]])
    
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        ax=axes[idx],
        cbar=True,
        square=True
    )
    
    axes[idx].set_xlabel('Predicted', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=12, fontweight='bold')
    axes[idx].set_title(
        f'{seg_id}\n{result["algorithm"]}',
        fontsize=12, fontweight='bold'
    )
    axes[idx].set_xticklabels(['No Churn', 'Churn'])
    axes[idx].set_yticklabels(['No Churn', 'Churn'])

plt.tight_layout()
cm_path = config.FIGURES_DIR / 'final_confusion_matrices.png'
plt.savefig(cm_path, dpi=config.FIGURE_DPI, bbox_inches='tight')
plt.show()

print(f"\n✓ Сохранено: {cm_path}")
print("="*80)

---
# 8. СОХРАНЕНИЕ ФИНАЛЬНЫХ МОДЕЛЕЙ

In [ ]:
# ====================================================================================
# СОХРАНЕНИЕ ФИНАЛЬНЫХ МОДЕЛЕЙ
# ====================================================================================

print("\n" + "="*80)
print("СОХРАНЕНИЕ ФИНАЛЬНЫХ МОДЕЛЕЙ")
print("="*80)

for seg_id, model_data in final_models.items():
    seg_num = seg_id.split()[1]
    algorithm = final_results[seg_id]['algorithm']
    
    # Имя файла
    algo_name = algorithm.lower().replace(' ', '_')
    model_filename = f"final_model_seg{seg_num}_{algo_name}.pkl"
    model_path = config.MODELS_DIR / model_filename
    
    # Сохраняем модель
    with open(model_path, 'wb') as f:
        pickle.dump(model_data['model'], f)
    
    file_size = model_path.stat().st_size / 1024
    
    print(f"\n✓ {model_filename}")
    print(f"  Сегмент: {seg_id}")
    print(f"  Алгоритм: {algorithm}")
    print(f"  ROC-AUC: {final_results[seg_id]['roc_auc']:.4f}")
    print(f"  Размер: {file_size:.2f} KB")

print("\n" + "="*80)
print("✓ Все модели сохранены")
print("="*80)

In [ ]:
# ====================================================================================
# СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
# ====================================================================================

print("\n" + "="*80)
print("СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*80)

# Создаем DataFrame с результатами
results_list = []
for seg_id, result in final_results.items():
    results_list.append({
        'segment': seg_id,
        'segment_name': config.SEGMENTS[seg_id]['name'],
        **result
    })

results_df = pd.DataFrame(results_list)

# Сохраняем
results_file = config.OUTPUT_DIR / 'final_production_models_results.csv'
results_df.to_csv(results_file, index=False)

print(f"\n✓ Результаты сохранены: {results_file}")
print(f"\nИтоговая таблица:")
print(results_df.to_string(index=False))

print("\n" + "="*80)

---
# 9. ФИНАЛЬНАЯ СВОДКА

In [ ]:
# ====================================================================================
# ФИНАЛЬНАЯ СВОДКА
# ====================================================================================

print("\n\n" + "="*80)
print("✓✓✓ ФИНАЛЬНЫЕ PRODUCTION МОДЕЛИ ГОТОВЫ ✓✓✓")
print("="*80)

print(f"\nДата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Random seed: {config.RANDOM_SEED}")

print(f"\n{'='*80}")
print("ФИНАЛЬНЫЕ МОДЕЛИ")
print(f"{'='*80}")

for seg_id, result in final_results.items():
    print(f"\n{seg_id}: {config.SEGMENTS[seg_id]['name']}")
    print("-" * 80)
    print(f"  Алгоритм: {result['algorithm']}")
    print(f"  Балансировка: {result['balancing_method']}")
    print(f"  ROC-AUC: {result['roc_auc']:.4f}")
    print(f"  Gini: {result['gini']:.4f}")
    print(f"  F1-Score: {result['f1']:.4f}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  Threshold: {result['threshold']:.4f}")
    print(f"  Время обучения: {result['train_time']:.2f} сек")

print(f"\n{'='*80}")
print("СОХРАНЕННЫЕ ФАЙЛЫ")
print(f"{'='*80}")

print(f"\nМОДЕЛИ (models/):")
for seg_id, result in final_results.items():
    seg_num = seg_id.split()[1]
    algo_name = result['algorithm'].lower().replace(' ', '_')
    print(f"  • final_model_seg{seg_num}_{algo_name}.pkl")

print(f"\nРЕЗУЛЬТАТЫ (output/):")
print(f"  • final_production_models_results.csv")
for seg_id in final_results.keys():
    seg_num = seg_id.split()[1]
    print(f"  • psi_analysis_seg{seg_num}.csv")
    print(f"  • target_correlation_seg{seg_num}.csv")
    print(f"  • shap_importance_seg{seg_num}.csv")

print(f"\nВИЗУАЛИЗАЦИИ (figures/):")
print(f"  • model_comparison_roc_gini.png       (Section 4.2 - Обоснование выбора)")
for seg_id in final_results.keys():
    seg_num = seg_id.split()[1]
    print(f"  • correlation_matrix_seg{seg_num}.png      (Section 3.5.4)")
print(f"  • final_roc_curves.png")
print(f"  • final_confusion_matrices.png")
for seg_id in final_results.keys():
    seg_num = seg_id.split()[1]
    print(f"  • shap_importance_seg{seg_num}.png         (Section 5.3)")
    print(f"  • shap_beeswarm_seg{seg_num}.png           (Section 5.3)")

print(f"\n{'='*80}")
print("ПОКРЫТИЕ ДОКУМЕНТАЦИИ")
print(f"{'='*80}")
print("\n✓ Section 3.3:   Статистика ABT (количество наблюдений, признаков, target rate)")
print("✓ Section 3.5.3: PSI Index (Train vs Test stability analysis)")
print("✓ Section 3.5.4: Корреляционный анализ и мультиколлинеарность")
print("✓ Section 4.2:   Сравнение моделей (ROC-AUC/Gini visualization)")
print("✓ Section 5.3:   SHAP Analysis (Feature Importance + Explainability)")

print(f"\n{'='*80}")
print("REPRODUCIBILITY")
print(f"{'='*80}")
print(f"\n✓ Random seed зафиксирован: {config.RANDOM_SEED}")
print(f"✓ Все параметры моделей зафиксированы")
print(f"✓ Балансировка использует фиксированный seed")
print(f"✓ Лучшие модели прописаны в коде (не зависит от experiments_all.csv)")
print(f"✓ Требуются только parquet файлы из output/")
print(f"✓ Run All должен давать идентичные результаты")

print(f"\n{'='*80}")
print("✓ ГОТОВО К ВАЛИДАЦИИ")
print(f"{'='*80}")